In [2]:
## Install the required packages
!pip install -qU elasticsearch openai

In [10]:
import os
from elasticsearch import Elasticsearch
from openai import OpenAI

In [11]:

es_client = Elasticsearch(
    "https://0fff4a3adf484903be8f4465eb8ffd6c.us-central1.gcp.cloud.es.io:443",
    api_key="bVdQVVRwRUI4WDhGRkpjMkxCM0M6SmMxbERTS2dTUmV6VWp5dHhPNzZNdw=="
)
      
openai_client = OpenAI(
    api_key="sk-svcacct-U71IhDCakJOnIBYwYIe6yF2AWEBqra5eR8is0mMmZ_nNf5JL_IqLT3BlbkFJIZ4wzAtNEGhKaUFAU9cwASyea_9Bh9x940xN8yc0hrQgkxjq4rUA",
)
index_source_fields = {
    "search-wwwdavitacom": [
        "body_content"
    ]
}

def get_elasticsearch_results(query):
    es_query = {
        "retriever": {
            "standard": {
                "query": {
                    "multi_match": {
                        "query": query,
                        "fields": [
                            "title",
                            "body_content"
                        ]
                    }
                }
            }
        },
        "size": 3
    }
    result = es_client.search(index="search-wwwdavitacom", body=es_query)
    return result["hits"]["hits"]



In [12]:
question = "can you pull recommended information for CKD patient medications"
elasticsearch_results = get_elasticsearch_results(question)

In [13]:
def create_openai_prompt(question, results):
    context = ""
    for hit in results:
      source_field = index_source_fields.get(hit["_index"])[0]
      hit_context = hit["_source"][source_field]
      context += f"{hit_context}\n"
    prompt = f"""
  Instructions:
  
  - You are an assistant for question-answering tasks.
  - Answer questions truthfully and factually using only the information presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer!
  
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.
  
  Context:
  {context}
  Question: {question}
  Answer:
  """
    return prompt


In [14]:
context_prompt = create_openai_prompt(question, elasticsearch_results)
#context_prompt

In [15]:
def generate_openai_completion(user_prompt):
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant for question-answering tasks."},
            {"role": "user", "content": user_prompt},
        ]
    )
    return response.choices[0].message.content


In [16]:
openai_completion = generate_openai_completion(context_prompt)
print(openai_completion)

I am unable to find specific information on recommended medications for CKD patients in the provided text.
